In [672]:
import numpy as np
import polars as pl

In [673]:
def get_lf(path: str):
    return pl.scan_csv(path, separator=";")


entities = get_lf("data/data_for_spb_hakaton_entities1-Table 1.csv").filter(pl.col("entity_id").is_not_null()).select(
    "entity_id", "area", "type", "status", "priority", "ticket_number", "name", "estimation").collect()

ticket2id = dict(zip(entities["ticket_number"].to_list(), entities["entity_id"].to_list()))

entities = entities.select("entity_id", "area", "type", "status", "priority", "name", "estimation").with_columns(
    pl.col('status').replace_strict(
        ['Создано', 'Закрыто', 'Выполнено', 'Отклонен исполнителем'],
        [0, 2, 2, 2], default=1, return_dtype=pl.Int8)).with_columns(
    pl.col('type').replace_strict(
        ['Дефект'],
        [1], default=0, return_dtype=pl.Int8)
).with_columns(pl.col("estimation").fill_null(0)).with_columns(pl.lit(0).alias("status"))
entities

entity_id,area,type,status,priority,name,estimation
i64,str,i8,i32,str,str,i64
94297,"""Система.Таск-трекер""",1,0,"""Средний""","""[FE] Бэклог. Кастомизация коло…",60
102481,"""Система.Ошибки""",0,0,"""Критический""","""[ГенераторДокументов] Интеграц…",432000
1805925,"""Система.Таск-трекер""",1,0,"""Высокий""","""[FE] История изменений. Пустые…",60
1934905,"""Система.Таск-трекер""",1,0,"""Средний""","""[FE] Зависимые поля. Тип реакц…",0
1943849,"""Система.Ошибки""",1,0,"""Низкий""","""[BE] При сортировке по Теме ес…",0
…,…,…,…,…,…,…
5179477,"""Управление релизами изменениям…",0,0,"""Средний""","""[PPPL] [FE] - Исследовать ui-k…",57600
5179714,"""Управление релизами изменениям…",0,0,"""Средний""","""[PPPL] [FE] - Написать базовую…",86400
5179793,"""Управление релизами изменениям…",0,0,"""Средний""","""[PPPL] [FE] - Сделать роутинг…",28800


In [683]:
import numpy as np

In [688]:
np.array([1,2,3]).tolist()

[1, 2, 3]

In [680]:
sprints = get_lf("data/sprints-Table 1.csv").filter(pl.col("sprint_name").is_not_null()).select(
    "sprint_name", "sprint_start_date", "entity_ids").with_columns(
    pl.col("sprint_start_date").str.to_datetime().dt.timestamp("ms") // 86400000).collect()

tasks = sprints.with_columns(pl.col("entity_ids").str.strip_chars(r'{}').str.split(",").alias("entity_id"))
t = tasks.with_columns(pl.col("entity_id").list.len())
         
tasks = tasks.explode("entity_id").with_columns(pl.col("entity_id").cast(pl.Int64, strict=False))
all_tasks_estimations = entities.select("entity_id", "estimation").join(tasks.select("entity_id", "sprint_name"), on="entity_id").group_by("sprint_name").agg(pl.col("estimation").sum())


sprints = {k: (i, v) for i, (k, v) in
           enumerate(zip(sprints["sprint_name"].to_list(), sprints["sprint_start_date"].to_list()))}

from enum import IntEnum


class Feature(IntEnum):
    to_do_estimation = 0
    processed_estimation = 1
    done_estimation = 2
    removed_estimation = 3
    blocked_tasks_estimation = 4
    created_tasks_estimation = 5
    excluded_tasks_estimation = 6
    created_tasks_amount = 7
    excluded_tasks_amount = 8


day_features = np.zeros((len(sprints), 14, len(Feature._member_names_)))

In [689]:
all_tasks_estimations

sprint_name,estimation
str,i64
"""Спринт 2024.3.5.NPP Shared Spr…",17784060
"""Спринт 2024.3.3.NPP Shared Spr…",17776920
"""Спринт 2024.3.4.NPP Shared Spr…",15811200
"""Спринт 2024.3.1.NPP Shared Spr…",18338580
"""Спринт 2024.3.2.NPP Shared Spr…",17067720
"""Спринт 2024.3.6.NPP Shared Spr…",20214060


In [681]:
t

sprint_name,sprint_start_date,entity_ids,entity_id
str,i64,str,u32
"""Спринт 2024.3.1.NPP Shared Spr…",19907,"""{4449728,4450628,4451563,44519…",464
"""Спринт 2024.3.2.NPP Shared Spr…",19921,"""{4506286,4429413,4327418,43700…",437
"""Спринт 2024.3.3.NPP Shared Spr…",19935,"""{4646403,4176602,4555571,44974…",501
"""Спринт 2024.3.4.NPP Shared Spr…",19949,"""{4805777,4596004,4594168,45237…",375
"""Спринт 2024.3.5.NPP Shared Spr…",19963,"""{4861060,4856927,4868451,40926…",540
"""Спринт 2024.3.6.NPP Shared Spr…",19977,"""{5034080,5052223,4620977,51371…",513


In [682]:
all_tasks_estimations

sprint_name,estimation
str,i64
"""Спринт 2024.3.5.NPP Shared Spr…",17784060
"""Спринт 2024.3.3.NPP Shared Spr…",17776920
"""Спринт 2024.3.4.NPP Shared Spr…",15811200
"""Спринт 2024.3.1.NPP Shared Spr…",18338580
"""Спринт 2024.3.2.NPP Shared Spr…",17067720
"""Спринт 2024.3.6.NPP Shared Spr…",20214060


In [641]:
# entities
state2int = {"created": 0, "closed": 2, "done": 2, "rejectedByThePerformer": 2}
entities = {k: list(v[0]) + ["", "", 0] for k, v in entities.rows_by_key("entity_id").items()}


In [645]:
sprints["Спринт 2024.3.1.NPP Shared Sprint"]

(0, 19907)

In [646]:
entities[4533564]

['Система.Таск-трекер',
 0,
 2,
 'Средний',
 '[BE] Добавление типов полей в таблицы - Поддержка экспорта',
 0,
 '',
 '',
 0]

In [647]:
entities

{94297: ['Система.Таск-трекер',
  1,
  2,
  'Средний',
  '[FE] Бэклог. Кастомизация колонок. Кастомизация для панелей "спринты" и  "бэклоги" зависима друг от друга',
  60,
  '',
  '',
  0],
 102481: ['Система.Ошибки',
  0,
  2,
  'Критический',
  '[ГенераторДокументов] Интеграция со Система.ГенераторДокументов в части синхронизации Стандартных решений',
  432000,
  '',
  '',
  0],
 1805925: ['Система.Таск-трекер',
  1,
  1,
  'Высокий',
  '[FE] История изменений. Пустые строки в истории изменений, если выбрать значение, которое уже было выбрано до редактирования поля',
  60,
  '',
  '',
  0],
 1934905: ['Система.Таск-трекер',
  1,
  2,
  'Средний',
  '[FE] Зависимые поля. Тип реакции disable НЕ работает для некоторых типов полей',
  0,
  '',
  '',
  0],
 1943849: ['Система.Ошибки',
  1,
  2,
  'Низкий',
  '[BE] При сортировке по Теме если знаки препинания стоят перед буквенными значениями сортировка отрабатывает неправильно',
  0,
  '',
  '',
  0],
 1966759: ['Управление релизами измен

In [648]:
history = get_lf("data/history-Table 1.csv").filter(
    pl.col("entity_id").is_not_null()
    & pl.col("history_property_name").is_not_null()
    & pl.col("history_property_name").is_in(["Статус", "Резолюция", "Спринт", "Связанные Задачи"])
).select(
    "entity_id", "history_property_name", "history_date", "history_change").collect()
history = history.with_columns(
    pl.col("history_date").str.to_datetime('%m/%d/%y %H:%M', time_unit="ms", strict=False)).with_columns(
    pl.col("history_change").str.split(" -> ").list.last()).sort(
    'history_date', maintain_order=True).with_columns(
    pl.col('history_change').shift(1).alias('history_change_prev')).with_columns(
    pl.col('entity_id').shift(1).alias('entity_id_prev')).with_columns(
    (pl.col("history_date").dt.timestamp("ms") // 86400000).alias("day"))

In [649]:
history.filter(pl.col("history_property_name") == "Резолюция")["history_change"].unique().to_list()

['Готово', 'Отклонено', None, 'Дубликат', 'Отменен инициатором']

In [650]:
history["history_property_name"].unique().to_list()

['Связанные Задачи', 'Статус', 'Спринт', 'Резолюция']

In [651]:
history

entity_id,history_property_name,history_date,history_change,history_change_prev,entity_id_prev,day
i64,str,datetime[ms],str,str,i64,i64
1805925,"""Спринт""",2023-07-12 13:29:00,"""Спринт 2023.3.2.NPP Shared Spr…",null,null,19550
1805925,"""Спринт""",2023-07-31 17:43:00,null,"""Спринт 2023.3.2.NPP Shared Spr…",1805925,19569
1805925,"""Спринт""",2023-07-31 17:43:00,"""Спринт 2023.3.3.NPP Shared Spr…",null,1805925,19569
1934905,"""Связанные Задачи""",2023-08-04 11:32:00,"""8""","""Спринт 2023.3.3.NPP Shared Spr…",1805925,19573
1934905,"""Связанные Задачи""",2023-08-04 11:32:00,"""PPTS-3254""","""8""",1934905,19573
…,…,…,…,…,…,…
4707481,"""Спринт""",2024-11-06 08:05:00,"""Спринт 2024.3.6.NPP Shared Spr…","""Спринт 2024.3.4.NPP Shared Spr…",4707481,20033
4707481,"""Спринт""",2024-11-06 08:05:00,"""Спринт 2024.4.1.NPP Shared Spr…","""Спринт 2024.3.6.NPP Shared Spr…",4707481,20033
4707481,"""Спринт""",2024-11-06 08:05:00,"""Спринт 2024.4.2.NPP Shared Spr…","""Спринт 2024.4.1.NPP Shared Spr…",4707481,20033


In [652]:
history.filter(pl.col("history_property_name") == "Статус")

entity_id,history_property_name,history_date,history_change,history_change_prev,entity_id_prev,day
i64,str,datetime[ms],str,str,i64,i64
102481,"""Статус""",2023-10-23 15:52:00,"""analysis""",null,2441405,19653
102481,"""Статус""",2023-10-24 15:27:00,"""design""","""Спринт 2023.4.3.NPP Shared Spr…",2444222,19654
102481,"""Статус""",2023-10-24 15:27:00,"""analysis""","""design""",102481,19654
102481,"""Статус""",2023-10-30 14:13:00,"""design""","""analysis""",102481,19660
102481,"""Статус""",2023-10-30 14:13:00,"""readyForDevelopment""","""design""",102481,19660
…,…,…,…,…,…,…
4600573,"""Статус""",2024-11-02 07:01:00,"""testing""","""verification""",4914666,20029
4956270,"""Статус""",2024-11-02 08:05:00,"""analysis""","""testing""",4600573,20029
4681026,"""Статус""",2024-11-02 11:33:00,"""testing""","""analysis""",4956270,20029


In [653]:
entities

{94297: ['Система.Таск-трекер',
  1,
  2,
  'Средний',
  '[FE] Бэклог. Кастомизация колонок. Кастомизация для панелей "спринты" и  "бэклоги" зависима друг от друга',
  60,
  '',
  '',
  0],
 102481: ['Система.Ошибки',
  0,
  2,
  'Критический',
  '[ГенераторДокументов] Интеграция со Система.ГенераторДокументов в части синхронизации Стандартных решений',
  432000,
  '',
  '',
  0],
 1805925: ['Система.Таск-трекер',
  1,
  1,
  'Высокий',
  '[FE] История изменений. Пустые строки в истории изменений, если выбрать значение, которое уже было выбрано до редактирования поля',
  60,
  '',
  '',
  0],
 1934905: ['Система.Таск-трекер',
  1,
  2,
  'Средний',
  '[FE] Зависимые поля. Тип реакции disable НЕ работает для некоторых типов полей',
  0,
  '',
  '',
  0],
 1943849: ['Система.Ошибки',
  1,
  2,
  'Низкий',
  '[BE] При сортировке по Теме если знаки препинания стоят перед буквенными значениями сортировка отрабатывает неправильно',
  0,
  '',
  '',
  0],
 1966759: ['Управление релизами измен

In [654]:
sprints

{'Спринт 2024.3.1.NPP Shared Sprint': (0, 19907),
 'Спринт 2024.3.2.NPP Shared Sprint': (1, 19921),
 'Спринт 2024.3.3.NPP Shared Sprint': (2, 19935),
 'Спринт 2024.3.4.NPP Shared Sprint': (3, 19949),
 'Спринт 2024.3.5.NPP Shared Sprint': (4, 19963),
 'Спринт 2024.3.6.NPP Shared Sprint': (5, 19977)}

In [655]:
h.filter(pl.col("entity_id")==3941779)

entity_id,history_property_name,history_date,history_change,history_change_prev,entity_id_prev,day
i64,str,datetime[ms],str,str,i64,i64
3941779,"""Связанные Задачи""",2024-05-08 10:54:00,"""isClonedBy""","""Спринт 2024.2.4.NPP Shared Spr…",2623949,19851
3941779,"""Связанные Задачи""",2024-05-08 10:54:00,"""PPAR-6588""","""isClonedBy""",3941779,19851
3941779,"""Связанные Задачи""",2024-05-08 10:55:00,"""childdest""","""PPAR-6588""",3941779,19851
3941779,"""Связанные Задачи""",2024-05-08 10:55:00,"""PPAR-6588""","""childdest""",3941779,19851
3941779,"""Связанные Задачи""",2024-05-08 10:55:00,null,"""PPAR-6588""",3941779,19851
…,…,…,…,…,…,…
3941779,"""Спринт""",2024-09-10 13:18:00,"""Спринт 2024.3.6.NPP Shared Spr…","""Спринт 2024.3.6.NPP Shared Spr…",2353124,19976
3941779,"""Резолюция""",2024-09-19 15:48:00,"""Готово""","""closed""",2353124,19985
3941779,"""Статус""",2024-09-19 15:48:00,"""closed""","""Готово""",3941779,19985


In [656]:
for (day,), df in history.group_by("day", maintain_order=True):
    df = df.with_columns(
        history_property_name=pl.when(pl.col("history_property_name")=="Статус").then(
            pl.col("history_change").replace_strict(
                ['created', 'closed', 'done', 'rejectedByThePerformer'],
            ['0', '2', '2', '2'], default='1', return_dtype=pl.String)
        ).otherwise(
            pl.col("history_property_name")
        )
    )
    df = df.group_by(["entity_id", "history_property_name"], maintain_order=True).last()
    for row in df.iter_rows():
        ent_id = row[0]
        if ent_id not in entities:
            continue
        ent = entities[ent_id]
        sprint_name = ent[7]
        sprint = sprints.get(sprint_name, None)
        day_offset = 15
        sprint_id = -1
        if sprint is not None:
            sprint_id = sprint[0]
            day_offset = sprint[1] - day
            if day_offset < 0:
                day_offset = 0
        # print(row)
        match row[1]:
            case "Резолюция":
                entities[ent_id][6] = row[3]
            case "0":
                print(row)
                ent[2] = 0
                if sprint_id!=-1 and day_offset < 14:
                    day_features[sprint_id, day_offset, Feature.to_do_estimation] += ent[5]
            case "1":
                ent[2] = 1
                if sprint_id!=-1 and day_offset < 14:
                    day_features[sprint_id, day_offset, Feature.processed_estimation] += ent[5]
            case "2":
                ent[2] = 2
                if sprint_id!=-1 and day_offset < 14:
                    if ent[6] in ['Отклонено', 'Дубликат', 'Отменен инициатором']:
                        day_features[sprint_id, day_offset, Feature.removed_estimation] += ent[5]
                    else:
                        day_features[sprint_id, day_offset, Feature.done_estimation] += ent[5]
            case "Спринт":
                new_sprint_name = row[3]
                
                old_sprint_name = ent[7]
                ent[7] = new_sprint_name                
                new_sprint = sprints.get(new_sprint_name, None)
                old_sprint = sprints.get(old_sprint_name, None)
                if new_sprint:
                    # print(ent[2])
                    day_offset = new_sprint[1] - day
                    sprint_id = new_sprint[0]
                    if day_offset < 0:
                        day_offset = 0
                    if day_offset < 14:
                        day_features[sprint_id, day_offset, Feature.created_tasks_estimation] += ent[5]
                        day_features[sprint_id, day_offset, Feature.created_tasks_amount] += 1
                        match ent[2]:
                            case 0:
                                day_features[sprint_id, day_offset, Feature.to_do_estimation] += ent[5]
                            case 1:
                                day_features[sprint_id, day_offset, Feature.processed_estimation] += ent[5]
                            case 2:
                                if ent[6] in ['Отклонено', 'Дубликат', 'Отменен инициатором']:
                                    day_features[sprint_id, day_offset, Feature.removed_estimation] += ent[5]
                                else:
                                    day_features[sprint_id, day_offset, Feature.done_estimation] += ent[5]
                            
                        day_features[sprint_id, day_offset, Feature.created_tasks_amount] += 1
                        
                        if ent[8]!=0:
                            blocked_by = entities.get(ent[8], None)
                            if blocked_by and blocked_by[2] !=2:
                                day_features[sprint_id, day_offset, Feature.blocked_tasks_estimation] += blocked_by[5]
                
                if old_sprint:
                    day_offset = old_sprint[1] - day
                    sprint_id = old_sprint[0]
                    if day_offset < 0:
                        day_offset = 0
                    if day_offset < 14:
                        # print(f"remove from {sprint_id} at day {day_offset}")
                        day_features[sprint_id, day_offset, Feature.excluded_tasks_estimation] += ent[5]
                        day_features[sprint_id, day_offset, Feature.excluded_tasks_amount] += 1
            case 'Связанные Задачи':
                if row[3]=="":
                    ent[8] = 0
                else:
                    if row[4]=="isBlockedBy":
                        blocked_by_id = ticket2id.get(row[3],0)
                        ent[8] = blocked_by_id
                        blocked_by = entities.get(blocked_by_id, None)
                        if blocked_by and blocked_by[3]!=2:
                            sprint_name = ent[7]
                            sprint = sprints.get(sprint_name, None)
                            if sprint is not None:
                                sprint_id = sprint[0]
                                day_offset = sprint[1] - day
                                if day_offset < 0:
                                    day_offset = 0
                                elif day_offset > 13:
                                    continue
                                day_features[sprint_id, day_offset, Feature.blocked_tasks_estimation] += blocked_by[5]

(102481, '0', datetime.datetime(2024, 2, 13, 15, 14), 'created', 'Спринт 2024.1.4.NPP Shared Sprint', 2623949, 19766)
(3864586, '0', datetime.datetime(2024, 5, 20, 10, 12), 'created', 'inProgress', 3864586, 19863)
(3907488, '0', datetime.datetime(2024, 5, 22, 8, 5), 'created', 'inProgress', 3907488, 19865)
(4317978, '0', datetime.datetime(2024, 6, 21, 7, 30), 'created', 'inProgress', 4317978, 19895)
(4317979, '0', datetime.datetime(2024, 6, 21, 7, 30), 'created', 'created', 4317978, 19895)
(4319076, '0', datetime.datetime(2024, 7, 2, 17, 15), 'created', 'Спринт 2024.3.1.NPP Shared Sprint', 4334966, 19906)
(4436271, '0', datetime.datetime(2024, 7, 3, 7, 40), 'created', 'Спринт 2024.3.1.NPP Shared Sprint', 4410789, 19907)
(4436329, '0', datetime.datetime(2024, 7, 3, 8, 27), 'created', 'PPWI-5375', 4449638, 19907)
(4448947, '0', datetime.datetime(2024, 7, 3, 9, 21), 'created', None, 4065300, 19907)
(4364703, '0', datetime.datetime(2024, 7, 4, 7, 16), 'created', 'PPAR-7420', 4340645, 19908

In [666]:
day_features[0]

array([[9.360000e+05, 1.857972e+07, 1.429572e+07, 1.872000e+05,
        0.000000e+00, 8.020860e+06, 4.777380e+06, 4.240000e+02,
        1.430000e+02],
       [4.320000e+04, 1.580580e+06, 2.491200e+06, 1.440000e+04,
        0.000000e+00, 4.071780e+06, 7.200000e+03, 2.600000e+02,
        1.000000e+00],
       [0.000000e+00, 5.760000e+04, 4.680000e+05, 0.000000e+00,
        0.000000e+00, 5.256000e+05, 0.000000e+00, 2.200000e+01,
        0.000000e+00],
       [0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00],
       [0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00],
       [0.000000e+00, 1.446000e+04, 2.448000e+05, 0.000000e+00,
        0.000000e+00, 2.304600e+05, 9.504000e+05, 2.000000e+01,
        2.000000e+00],
       [0.000000e+00, 1.440000e+05, 2.880000e+04, 0.000000e+00,
        0.000000e+00, 1.728000

In [658]:
Feature.blocked_tasks_estimation

<Feature.blocked_tasks_estimation: 4>

In [659]:
history = history.with_columns(pl.col("history_change").str.split(" -> ").list.last())
# history
history

entity_id,history_property_name,history_date,history_change,history_change_prev,entity_id_prev,day
i64,str,datetime[ms],str,str,i64,i64
1805925,"""Спринт""",2023-07-12 13:29:00,"""Спринт 2023.3.2.NPP Shared Spr…",null,null,19550
1805925,"""Спринт""",2023-07-31 17:43:00,null,"""Спринт 2023.3.2.NPP Shared Spr…",1805925,19569
1805925,"""Спринт""",2023-07-31 17:43:00,"""Спринт 2023.3.3.NPP Shared Spr…",null,1805925,19569
1934905,"""Связанные Задачи""",2023-08-04 11:32:00,"""8""","""Спринт 2023.3.3.NPP Shared Spr…",1805925,19573
1934905,"""Связанные Задачи""",2023-08-04 11:32:00,"""PPTS-3254""","""8""",1934905,19573
…,…,…,…,…,…,…
4707481,"""Спринт""",2024-11-06 08:05:00,"""Спринт 2024.3.6.NPP Shared Spr…","""Спринт 2024.3.4.NPP Shared Spr…",4707481,20033
4707481,"""Спринт""",2024-11-06 08:05:00,"""Спринт 2024.4.1.NPP Shared Spr…","""Спринт 2024.3.6.NPP Shared Spr…",4707481,20033
4707481,"""Спринт""",2024-11-06 08:05:00,"""Спринт 2024.4.2.NPP Shared Spr…","""Спринт 2024.4.1.NPP Shared Spr…",4707481,20033


In [660]:
history.filter(~pl.col("history_change").str.contains("->"))

entity_id,history_property_name,history_date,history_change,history_change_prev,entity_id_prev,day
i64,str,datetime[ms],str,str,i64,i64
1805925,"""Спринт""",2023-07-12 13:29:00,"""Спринт 2023.3.2.NPP Shared Spr…",null,null,19550
1805925,"""Спринт""",2023-07-31 17:43:00,"""Спринт 2023.3.3.NPP Shared Spr…",null,1805925,19569
1934905,"""Связанные Задачи""",2023-08-04 11:32:00,"""8""","""Спринт 2023.3.3.NPP Shared Spr…",1805925,19573
1934905,"""Связанные Задачи""",2023-08-04 11:32:00,"""PPTS-3254""","""8""",1934905,19573
102481,"""Связанные Задачи""",2023-08-17 14:11:00,"""29""","""PPTS-3254""",1934905,19586
…,…,…,…,…,…,…
4707481,"""Спринт""",2024-11-06 08:05:00,"""Спринт 2024.3.6.NPP Shared Spr…","""Спринт 2024.3.4.NPP Shared Spr…",4707481,20033
4707481,"""Спринт""",2024-11-06 08:05:00,"""Спринт 2024.4.1.NPP Shared Spr…","""Спринт 2024.3.6.NPP Shared Spr…",4707481,20033
4707481,"""Спринт""",2024-11-06 08:05:00,"""Спринт 2024.4.2.NPP Shared Spr…","""Спринт 2024.4.1.NPP Shared Spr…",4707481,20033
